** 주식 종목 필터링 **

1. 코스닥 데이터 크롤링 (Request, BeautifulSoup 사용)
2. 데이터에서 주당 배당금 Top 50 항목 추출 (Pandas nlargest Method 사용)
3. 부채자본 비율 <10 데이터 추출 (yfinance 사용)

In [1]:
import FinanceDataReader as fdr
import yfinance as yf
from pykrx import stock
import pandas as pd
import numpy as np

In [2]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2023.11.28


In [3]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20231128


In [5]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

WQBcysHjXXfpJJmvPlQ76JONmAQKSFONk4OGasuPzHERtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15efjSvElg2BRqxesJMIcURhMtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [6]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
gen_otp_ksq = {
    'searchType': '1',
    'mktId': 'KSQ',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,2290,-25,-1.08,30.0,76.33,NaN,NaN,947.0,2.42,0,0.00
1,054620,APS,6450,-50,-0.77,505.0,12.77,NaN,NaN,10864.0,0.59,0,0.00
2,265520,AP시스템,20500,100,0.49,5463.0,3.75,5685.0,3.61,17980.0,1.14,270,1.32
3,211270,AP위성,14680,-130,-0.88,287.0,51.15,671.0,21.87,5997.0,2.45,70,0.48
4,126600,BGF에코머티리얼즈,4110,-105,-2.49,1053.0,3.90,NaN,NaN,6585.0,0.62,100,2.43


In [11]:
KSQ = sector_ksq.nlargest(100,'주당배당금',keep='first')

In [17]:
KSQ_list = KSQ['종목코드'].tolist()

In [44]:
## Total Debt / Total SahreHolders Equity 필터링 

KSQ_list_update1 = [] ## 리스트로 저장
DebtToEquity = [] ##값 저장 (최종 결과 값에 열 추가)

for i in range(len(KSQ_list)):
    ticker = yf.Ticker(KSQ_list[i]+".KQ")
    info = ticker.info
    if 'debtToEquity' in info.keys():
        if(info['debtToEquity']<10):
            KSQ_list_update1.append(KSQ_list[i])
            DebtToEquity.append(info['debtToEquity'])
    else:
        continue


In [21]:
KSQ_list_update1

['034950',
 '065710',
 '003800',
 '053800',
 '039030',
 '049070',
 '100030',
 '089600',
 '143240',
 '036800',
 '192440',
 '040420',
 '348210',
 '136540',
 '053610',
 '234340',
 '241770',
 '340570',
 '214180',
 '066620',
 '319660']

In [45]:
DebtToEquity

[2.203,
 0.563,
 0.023,
 0.188,
 1.615,
 3.136,
 3.337,
 4.267,
 2.91,
 4.229,
 1.03,
 5.452,
 1.357,
 1.315,
 5.286,
 3.13,
 1.071,
 0.1,
 4.048,
 0.263,
 5.8]

In [31]:
## 주당배당금 Top 50 데이터에에서 Matching 되는 값을 찾기 위해 INDEX화
KSQ_index = []
for i in range(len(KSQ_list_update1)):
    KSQ_index.append(sector_ksq['종목코드'].tolist().index(KSQ_list_update1[i]))

In [37]:
KSQ_index

[1588,
 580,
 864,
 779,
 1158,
 1185,
 1181,
 168,
 519,
 172,
 642,
 1214,
 194,
 1076,
 1520,
 1640,
 404,
 1447,
 1639,
 135,
 1536]

In [52]:
final_previous = sector_ksq.iloc[KSQ_index,:]

In [56]:
final = final_previous.assign(부채자본비율 = DebtToEquity)

In [57]:
final

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,부채자본비율
1588,034950,한국기업평가,78500,200,0.26,4582.0,17.13,NaN,NaN,25180.0,3.12,5100,6.50,2.203
580,065710,서호전기,19000,-40,-0.21,2215.0,8.58,NaN,NaN,17842.0,1.06,1500,7.89,0.563
864,003800,에이스침대,26050,-150,-0.57,5293.0,4.92,NaN,NaN,58224.0,0.45,1330,5.11,0.023
779,053800,안랩,68300,100,0.15,1631.0,41.88,3464.0,19.72,29416.0,2.32,1100,1.61,0.188
1158,039030,이오테크닉스,162300,4100,2.59,6293.0,25.79,5793.0,28.02,43984.0,3.69,1000,0.62,1.615
1185,049070,인탑스,29400,300,1.03,6147.0,4.78,2372.0,12.39,38127.0,0.77,860,2.93,3.136
1181,100030,인지소프트,16300,140,0.87,1360.0,11.99,NaN,NaN,18065.0,0.90,720,4.42,3.337
168,089600,나스미디어,20550,0,0.00,2375.0,8.65,2346.0,8.76,18182.0,1.13,700,3.41,4.267
519,143240,사람인,16820,-70,-0.41,2656.0,6.33,NaN,NaN,15673.0,1.07,700,4.16,2.910
172,036800,나이스정보통신,21850,0,0.00,4358.0,5.01,3788.0,5.77,33633.0,0.65,650,2.97,4.229
